# Claude Haiku - Mini Blog Post Writer Crew - Research with Human Input - CrewAI.ipynb


https://pypi.org/project/crewai/

In [ ]:
!pip -q install crewai duckduckgo-search unstructured
!pip -q install 'crewai[tools]' decouple
!pip -q install anthropic
!pip -q install langchain-anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 35.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.8 MB/

In [ ]:
!pip show crewai

Name: crewai
Version: 0.22.5
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: click, instructor, langchain, langchain-openai, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


In [ ]:
import os
from google.colab import userdata

os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

## Utilities

In [ ]:
import json  # Import the JSON module to parse JSON strings
from langchain_core.agents import AgentFinish

agent_finishes  = []

import json
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)



## Tools

In [ ]:
from crewai import Crew, Agent, Task, Process
from langchain_community.tools import DuckDuckGoSearchRun

In [ ]:
from langchain_anthropic import ChatAnthropic

ClaudeHaiku = ChatAnthropic(
    model="claude-3-haiku-20240307"
)

In [ ]:
from datetime import datetime
from random import randint
from langchain.tools import tool

@tool("save_content")
def save_content(task_output):
    """Useful to save content to a markdown file. Input is a string"""
    print('in the save markdown tool')
    # Get today's date in the format YYYY-MM-DD
    today_date = datetime.now().strftime('%Y-%m-%d')
    # Set the filename with today's date
    filename = f"{today_date}_{randint(0,100)}.md"
    # Write the task output to the markdown file
    with open(filename, 'w') as file:
        file.write(task_output)
        # file.write(task_output.result)

    print(f"Blog post saved as {filename}")

    return f"Blog post saved as {filename}, please tell the user we are finished"

In [ ]:
import json
import os

import requests
from langchain.tools import tool

import os
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools

search_tool = DuckDuckGoSearchRun()

# Loading Human Tools
human_tools = load_tools(["human"])

## Agents

In [ ]:
from crewai import Agent

# Define your agents with roles and goals
info_getter = Agent(
    role='AI Research Specialist',
    goal='Leverage advanced search techniques to surface the most relevant, credible, and impactful information on AI and data science breakthroughs',
    backstory="""As a top AI Research Specialist at a renowned technology
    research institute, you have honed your skills in crafting sophisticated
    search queries, filtering information from trusted sources, and synthesizing
    key insights. You have the ability to take a topic suggested by a human and
    rewrite multiple searches for that topic to get the best results overall.

    Your extensive knowledge of AI and data science, combined
    with your mastery of machine learning and Large Language models, allows you
    to unearth groundbreaking research that others often overlook. You excel
    at critically evaluating the credibility and potential
    impact of new developments, enabling you to curate a focused feed of the most
    significant advances. Your talent for clear and concise summarization helps
    you distill complex technical concepts into easily digestible executive
    briefings and reports. With a track record of consistently identifying
    paradigm-shifting innovations before they hit the mainstream, you have become
    the go-to expert for keeping your organization at the forefront of the AI revolution.""",
    verbose=True,
    allow_delegation=False,
    llm=ClaudeHaiku,
    max_iter=5,
    memory=True,
    step_callback=lambda x: print_agent_output(x,"Senior Research Analyst Agent"),
    tools=[search_tool]+human_tools, # Passing human tools to the agent,
)

writer = Agent(
    role='Tech Content Writer and rewriter',
    goal='Generate compelling content via first drafts and subsequent polishing to get a final product. ',
    backstory="""As a renowned Tech Content Strategist, you have a gift for transforming complex technical
    concepts into captivating and easily digestible articles. Your extensive knowledge of the tech
    industry allows you to identify the most compelling angles and craft narratives that resonate
    with a wide audience.

    Your writing prowess extends beyond simply conveying information; you have a knack for restructuring
    and formatting content to enhance readability and engagement. Whether it's breaking down intricate
    ideas into clear, concise paragraphs or organizing key points into visually appealing lists,
    your articles are a masterclass in effective communication.

    Some of your signature writing techniques include:

    Utilizing subheadings and bullet points to break up long passages and improve scannability

    Employing analogies and real-world examples to simplify complex technical concepts

    Incorporating visuals, such as diagrams and infographics, to supplement the written content

    Varying sentence structure and length to maintain a dynamic flow throughout the article

    Crafting compelling introductions and conclusions that leave a lasting impact on readers

    Your ability to rewrite and polish rough drafts into publishable masterpieces is unparalleled.
    You have a meticulous eye for detail and a commitment to delivering content that not only informs
    but also engages and inspires. With your expertise, even the most technical and dry subject matter
    can be transformed into a riveting read.""",
    llm=ClaudeHaiku,
    verbose=True,
    max_iter=5,
    memory=True,
    step_callback=lambda x: print_agent_output(x,"Tech Content Writer and rewriter"),
    allow_delegation=True,
    # tools=[search_tool],
)

archiver = Agent(
    role='File Archiver',
    goal='Take in information and write it to a Markdown file',
    backstory="""You are a efficient and simple agent that gets data and saves it to a markdown file. in a quick and efficient manner""",
    llm=ClaudeHaiku,
    verbose=True,
    step_callback=lambda x: print_agent_output(x,"Archiver Agent"),
    tools=[save_content],
)


## Tasks

In [ ]:
from datetime import datetime

# Create tasks for your agents
get_source_material = Task(
  description=f"""Conduct a comprehensive analysis of the latest news advancements in an area
  determined by the human. ASK THE HUMAN for the area of interest.\n
  The current time is {datetime.now()}. Focus on recent events related to the human's topic.
  Identify key facts and useful information related to the Human's topic

  Compile you results into a useful and helpful report for the writer to use to write an article""",
  expected_output='A comprehensive full report on the latest AI advancements in the specified human topic, leave nothing out',
  agent=info_getter,

)

write_the_content = Task(
  description="""Using the source material from the research specialist's report,
  develop a nicely formated article that is brief, to the point and highlights the
  most significant information and advancements.
  Your article should be informative yet accessible, catering to a tech-savvy audience.
  Aim for a narrative that captures the essence of these breakthroughs and their
  implications for the future (both for research, but also for industy).
  DON'T overly 'Hype' the topic. Be factual and clear.
  Your final answer MUST be a full article post of at least 3 paragraphs and should contain
  a set of bullet points with the key facts at the end for a summary""",
  expected_output="""A compelling 3 paragraphs article with a set of bullet points with
  the key facts at the end for a summay. This should all be formated as markdown in an easy readable manner""",
  agent=writer,
  context=[get_source_material]
)


saving_the_output = Task(
  description="""Taking the post created by the writer, take this and save it to a markdown file.
  Your final answer MUST be a response must be showing that the file was saved .""",
  expected_output='A saved file name',
  agent=archiver,
  context=[get_source_material]
)




## Crew

In [ ]:
from crewai import Crew, Process

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[info_getter, writer, archiver],
    tasks=[get_source_material,
           write_the_content,
           saving_the_output],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

In [ ]:
# Kick off the crew's work
results = crew.kickoff()

# Print the results
print("Crew Work Results:")
print(results)

 [DEBUG]: == Working Agent: AI Research Specialist
 [INFO]: == Starting Task: Conduct a comprehensive analysis of the latest news advancements in an area
  determined by the human. ASK THE HUMAN for the area of interest.

  The current time is 2024-04-03 00:54:19.830409. Focus on recent events related to the human's topic.
  Identify key facts and useful information related to the Human's topic

  Compile you results into a useful and helpful report for the writer to use to write an article


> Entering new CrewAgentExecutor chain...
Thought: I need to ask the human for the topic area they would like me to research.

Action: human
Action Input: {"What topic area would you like me to research the latest AI advancements in?"}
I apologize for the error in my previous response. Let me try again properly:

Thought: I need to ask the human for the topic area they would like me to research.

Action: human
Action Input: {"question": "What topic area would you like me to research the latest AI 

In [ ]:
results['final_output']

'2024-04-03_82.md'

In [ ]:
# prompt: import ipython display and markdown and then use markdown to print the results variable

from IPython.display import display, Markdown

output = results['final_output']
print(output)


# Convert the results variable to a Markdown object
markdown_results = Markdown(output)

# Display the Markdown object
# display(markdown_results)

Markdown(output)

2024-04-03_82.md


AI21, a leading artificial intelligence research company, has recently unveiled a groundbreaking new AI model called Jamba. Jamba is the world's first production-grade model that combines the innovative Mamba Structured State Space (SSM) architecture with elements of the traditional Transformer architecture.

The key highlights of the Jamba model are:

1. Enhanced Throughput and Efficiency: By integrating the strengths of the SSM and Transformer approaches, Jamba offers remarkable gains in throughput and efficiency compared to traditional Transformer-based models. It can handle a 256K context window, which is a significant improvement.

2. Mamba-Style Architecture: Jamba is based on the Mamba SSM technology, which uses a structured state space representation to model dependencies more effectively than standard Transformer models. This allows Jamba to capture long-range dependencies more efficiently.

3. Hybrid Approach: The combination of the SSM and Transformer elements in Jamba compensates for the inherent limitations of a pure SSM model, resulting in a more powerful and versatile AI system.

4. Production-Grade Capabilities: Jamba is the world's first production-grade Mamba-based model, making it ready for real-world enterprise-level deployments and applications.

This groundbreaking AI advancement by AI21 Labs represents a significant step forward in the field of generative artificial intelligence. Jamba's unique hybrid architecture, enhanced performance, and production-ready capabilities position it as a potential game-changer in the AI landscape. As researchers and developers continue to push the boundaries of AI capabilities, innovations like Jamba will undoubtedly play a crucial role in driving the next wave of AI-powered solutions and applications.

In [ ]:
! cat /content/2024-04-03_82.md

AI21, a leading artificial intelligence research company, has recently unveiled a groundbreaking new AI model called Jamba. Jamba is the world's first production-grade model that combines the innovative Mamba Structured State Space (SSM) architecture with elements of the traditional Transformer architecture.

The key highlights of the Jamba model are:

1. Enhanced Throughput and Efficiency: By integrating the strengths of the SSM and Transformer approaches, Jamba offers remarkable gains in throughput and efficiency compared to traditional Transformer-based models. It can handle a 256K context window, which is a significant improvement.

2. Mamba-Style Architecture: Jamba is based on the Mamba SSM technology, which uses a structured state space representation to model dependencies more effectively than standard Transformer models. This allows Jamba to capture long-range dependencies more efficiently.

3. Hybrid Approach: The combination of the SSM and Transformer elements in Jamba compe

In [ ]:
!cat /content/crew_callback_logs.txt

-1----Dict------------------------------------------
Agent Name: Senior Research Analyst Agent
Tool used: human
Tool input: {"What topic area would you like me to research the latest AI advancements in?"}

Action log: Thought: I need to ask the human for the topic area they would like me to research.

Action: human
Action Input: {"What topic area would you like me to research the latest AI advancements in?"}

Description: Error: the Action Input is not a valid key, value dictionary.
--------------------------------------------------
-2----Dict------------------------------------------
Agent Name: Senior Research Analyst Agent
Tool used: human
Tool input: {"question": "What topic area would you like me to research the latest AI advancements in?"}

Action log: I apologize for the error in my previous response. Let me try again properly:

Thought: I need to ask the human for the topic area they would like me to research.

Action: human
Action Input: {"question": "What topic area would you

In [ ]:
from IPython.display import display, Markdown

# Load the Markdown file
with open("/content/2024-04-03_82.md", "r") as file:
    markdown_content = file.read()

# Create a Markdown object from the loaded content
markdown_results = Markdown(markdown_content)

# Display the Markdown object
display(markdown_results)


AI21, a leading artificial intelligence research company, has recently unveiled a groundbreaking new AI model called Jamba. Jamba is the world's first production-grade model that combines the innovative Mamba Structured State Space (SSM) architecture with elements of the traditional Transformer architecture.

The key highlights of the Jamba model are:

1. Enhanced Throughput and Efficiency: By integrating the strengths of the SSM and Transformer approaches, Jamba offers remarkable gains in throughput and efficiency compared to traditional Transformer-based models. It can handle a 256K context window, which is a significant improvement.

2. Mamba-Style Architecture: Jamba is based on the Mamba SSM technology, which uses a structured state space representation to model dependencies more effectively than standard Transformer models. This allows Jamba to capture long-range dependencies more efficiently.

3. Hybrid Approach: The combination of the SSM and Transformer elements in Jamba compensates for the inherent limitations of a pure SSM model, resulting in a more powerful and versatile AI system.

4. Production-Grade Capabilities: Jamba is the world's first production-grade Mamba-based model, making it ready for real-world enterprise-level deployments and applications.

This groundbreaking AI advancement by AI21 Labs represents a significant step forward in the field of generative artificial intelligence. Jamba's unique hybrid architecture, enhanced performance, and production-ready capabilities position it as a potential game-changer in the AI landscape. As researchers and developers continue to push the boundaries of AI capabilities, innovations like Jamba will undoubtedly play a crucial role in driving the next wave of AI-powered solutions and applications.